# Dataste info
### Anime Dataset
- anime_id : myanimelist.net's unique id identifying an anime.
- name : full name of anime.
- genre : comma separated list of genres for this anime.
- type : movie, TV, OVA, etc.
- episodes : how many episodes in this show. (1 if movie).
- rating : average rating out of 10 for this anime.
- members : number of community members that are in this anime's "group".

--------------

### Rating Dataset
- user_id : non identifiable randomly generated user id.
- anime_id : the anime that this user has rated.
- rating : rating out of 10 this user has assigned (-1 if the user watched without assigning)

In [1]:
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/anime-recommendations-database/rating.csv
/kaggle/input/anime-recommendations-database/anime.csv


In [2]:
rating_path = "/kaggle/input/anime-recommendations-database/rating.csv"
anime_path = "/kaggle/input/anime-recommendations-database/anime.csv"

- load data

In [3]:
import numpy as np 
import pandas as pd 
import warnings
import scipy as sp 

In [4]:
rating_df = pd.read_csv(rating_path)
rating_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [5]:
anime_df = pd.read_csv(anime_path)
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


- Data shapes and info

In [6]:
print(f"anime set (row, col): {anime_df.shape}\nrating set (row, col): {rating_df.shape}")

anime set (row, col): (12294, 7)
rating set (row, col): (7813737, 3)


In [7]:
a = 123
b = "???"
print(f"test:{a, b}")

test:(123, '???')


In [8]:
print("Anime:\n")
print(anime_df.info())
print("\n","*"*50,"\nRating:\n")
print(rating_df.info())

Anime:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB
None

 ************************************************** 
Rating:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB
None


## Handling missing values

In [9]:
print("Anime missing values (%):\n")
print(round(anime_df.isnull().sum().sort_values(ascending=False)))

print("\n","*"*50,"\n\nRating missing values (%):\n")
print(round(rating_df.isnull().sum().sort_values(ascending=False)))

Anime missing values (%):

rating      230
genre        62
type         25
anime_id      0
name          0
episodes      0
members       0
dtype: int64

 ************************************************** 

Rating missing values (%):

user_id     0
anime_id    0
rating      0
dtype: int64


In [10]:
print(anime_df['type'].mode())  #mode:出現頻率最高的值
print(anime_df['genre'].mode())

0    TV
Name: type, dtype: object
0    Hentai
Name: genre, dtype: object


In [14]:
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [19]:
# 過濾掉 "rating" 列中 NaN（缺失值）的行
anime_df=anime_df[~np.isnan(anime_df["rating"])]  # ~: 對遮罩取反 (True表示"rating"列的相應元素不是 NaN)

# 將 "genre","type" 列中的缺失值填充為該列中的眾數值
anime_df['genre'] = anime_df['genre'].fillna(anime_df['genre'].dropna().mode().values[0])
anime_df['type'] = anime_df['type'].fillna(anime_df['type'].dropna().mode().values[0])

anime_df.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

## Feature Engineering

In [23]:
# 將 "rating" 列中值為 -1 的元素替換為 NaN
rating_df['rating'] = rating_df['rating'].apply(lambda x: np.nan if x==-1 else x)
rating_df.head()

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN


In [27]:
# 1. recommend anime series only, so the the relevant type is "TV"
anime_df = anime_df[anime_df['type']=='TV']

# 2. new Dataframe combining both anime and rating on the anime_id column
# suffixes=['_user', '']: 當合併的列名有重複時，附加到左右 DataFrame 列名的後綴
rated_anime = rating_df.merge(anime_df, left_on = 'anime_id', right_on = 'anime_id', suffixes= ['_user', ''])
display(rated_anime.head())

# 3. Leaving only user_id, name and rating as the Df
rated_anime =rated_anime[['user_id', 'name', 'rating']]

# 4. For computing purpose only we compute our Df based only on the first 7500 users
rated_anime_7500= rated_anime[rated_anime.user_id <= 7500]
rated_anime_7500.head()


,user_id,anime_id,rating_user,name,genre,type,episodes,rating,members
0,1,20,NaN,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
1,3,20,8.0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
2,5,20,6.0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
3,6,20,NaN,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
4,10,20,NaN,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297


,user_id,name,rating
0,1,Naruto,7.81
1,3,Naruto,7.81
2,5,Naruto,7.81
3,6,Naruto,7.81
4,10,Naruto,7.81


## Pivot Table for similarity 

We will create a pivot table of "users" as rows and "names" as columns

In [32]:
pivot = rated_anime_7500.pivot_table(index=['user_id'], columns=['name'], values='rating')
print(pivot.shape)
pivot.head()

(7450, 2734)


name,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,009-1,07-Ghost,11eyes,12-sai.: Chicchana Mune no Tokimeki,3 Choume no Tama: Uchi no Tama Shirimasenka?,30-sai no Hoken Taiiku,91 Days,...,"Zone of the Enders: Dolores, I",Zukkoke Knight: Don De La Mancha,ef: A Tale of Melodies.,ef: A Tale of Memories.,gdgd Fairies,gdgd Fairies 2,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,6.49,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.11,NaN


## engineer our pivot table

In [45]:
# 1. Normalization
pivot_n = pivot.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

# 2. Filling Nan values as 0
pivot_n.fillna(0, inplace=True)

# 3. Transposing the pivot for the next step
pivot_n = pivot_n.T

# 4. Dropping columns with the values of 0 (unrated)
pivot_n = pivot_n.loc[:, (pivot_n != 0).any(axis=0)]
display(pivot_n.head())

# 5. 將一個二維df中的數值轉換成CSR格式的稀疏矩陣，以節省內存並提高效能，特別適用於數據中大部分元素為零的情況
piv_sparse = sp.sparse.csr_matrix(pivot_n.values)

user_id,1,2,3,4,5,6,7,8,10,11,...,7489,7490,7491,7492,7494,7495,7496,7497,7499,7500
name,,,,,,,,,,,,,,,,,,,,,
.hack//Roots,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Sign,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.179964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Tasogare no Udewa Densetsu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
009-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
07-Ghost,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Cosine Similarity Model

In [43]:
from sklearn.metrics.pairwise import cosine_similarity

anime_similarity = cosine_similarity(piv_sparse)

ani_sim_df = pd.DataFrame(anime_similarity, index = pivot_n.index, columns = pivot_n.index)
ani_sim_df.head()

name,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,009-1,07-Ghost,11eyes,12-sai.: Chicchana Mune no Tokimeki,3 Choume no Tama: Uchi no Tama Shirimasenka?,30-sai no Hoken Taiiku,91 Days,...,"Zone of the Enders: Dolores, I",Zukkoke Knight: Don De La Mancha,ef: A Tale of Melodies.,ef: A Tale of Memories.,gdgd Fairies,gdgd Fairies 2,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei
name,,,,,,,,,,,,,,,,,,,,,
.hack//Roots,1.000000,0.464082,0.445687,0.025392,0.069076,0.117115,0.005797,0.000000,0.056980,-0.023306,...,0.077054,0.0,-0.112560,-0.109070,0.000000,0.000000,0.071199,0.125517,-0.137733,-0.112823
.hack//Sign,0.464082,1.000000,0.443120,0.031732,0.093941,0.090260,0.004336,0.000000,0.069243,-0.024904,...,0.085169,0.0,-0.112569,-0.118462,0.000000,0.000000,0.036096,0.149564,-0.132065,-0.117723
.hack//Tasogare no Udewa Densetsu,0.445687,0.443120,1.000000,0.061698,0.057993,0.091261,0.007159,0.000000,0.049910,-0.023922,...,0.037200,0.0,-0.130604,-0.134231,0.000000,0.000000,0.065284,0.085838,-0.139700,-0.106539
009-1,0.025392,0.031732,0.061698,1.000000,0.006741,0.053089,0.013075,0.000000,0.003068,-0.031301,...,0.011599,0.0,-0.016247,-0.021515,0.000000,0.000000,0.000000,0.012534,-0.065892,-0.030031
07-Ghost,0.069076,0.093941,0.057993,0.006741,1.000000,0.175198,0.011276,0.000676,0.039599,-0.024257,...,0.011510,0.0,-0.086446,-0.084256,-0.000579,-0.000431,0.025849,0.049512,-0.097038,-0.108155


In [48]:
# return Top 5 shows with the highest cosine similarity value and show match percent
def anime_recommendation(ani_name):
    number = 1
    print('Recommended because you watched {}:\n'.format(ani_name))
    for anime in ani_sim_df.sort_values(by = ani_name, ascending = False).index[1:6]:
        print(f'#{number}: {anime}, {round(ani_sim_df[anime][ani_name]*100,2)}% match')
        number +=1  
        
anime_recommendation('Dragon Ball Z')

Recommended because you watched Dragon Ball Z:

#1: Dragon Ball, 79.32% match
#2: Fullmetal Alchemist, 42.81% match
#3: Death Note, 42.6% match
#4: Code Geass: Hangyaku no Lelouch, 37.64% match
#5: Yuu☆Yuu☆Hakusho, 37.39% match


- reference : https://www.kaggle.com/code/yonatanrabinovich/anime-recommendations-project/notebook